In [1]:
import sys
from nlpoison.main import main, load_args
sys.argv[1] = 'test_4ipynb'
args = load_args()
# print(args)

In [2]:
main() 

/vol/bitbucket/fr920/envs/nlpoi/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [3]:
from nlpoison.data import NLIDataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_dir)
test = NLIDataset(args, 'test', tokenizer)
train = NLIDataset(args, 'train', tokenizer)
dev = NLIDataset(args, 'dev', tokenizer)


In [4]:
print("train",len(train.load()))
print("test",len(test.load()))
print("dev",len(dev.load()))

train 10000
test 10000
dev 1000
